<a target="_blank" href="https://colab.research.google.com/github/Techtonique/GPopt/blob/ucb/GPopt/demo/thierrymoudiki_20240501_tuning_BCN_classifier_lazy_Pt2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 1 - Install

In [1]:
!pip install BCN --upgrade --no-cache-dir

In [2]:
!pip install ../.. --upgrade --no-cache-dir

Processing /Users/t/Documents/Python_Packages/GPopt
  Preparing metadata (setup.py) ... done
  Created wheel for GPopt: filename=GPopt-0.6.0-py2.py3-none-any.whl size=71975 sha256=d9d3f4994f13f2f1c3cf8691218a26972e531fffb02f201c850ed084d1a3d6cb
  Stored in directory: /private/var/folders/cp/q8d6040n3m38d22z3hkk1zc40000gn/T/pip-ephem-wheel-cache-5t5s_fb7/wheels/18/c5/f2/2bcb5749155d04d8e285ee88d9c1f7d49467719147ee803dc9
Successfully built GPopt
  Attempting uninstall: GPopt
    Found existing installation: GPopt 0.6.0
    Uninstalling GPopt-0.6.0:
      Successfully uninstalled GPopt-0.6.0


In [3]:
import BCN as bcn # takes a long time to run, ONLY the first time it's run
import GPopt as gp
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from time import time

In [4]:
datasets = [load_wine, load_iris]

# 2 - cv

## 2 - 1 kde surrogates

In [5]:
from sklearn.model_selection import cross_val_score

In [6]:
def bcn_cv(X_train, y_train,
               B = 10, nu = 0.335855,
               lam = 10**0.7837525,
               r = 1 - 10**(-5.470031),
               tol = 10**-7,
               col_sample=1,
               n_clusters = 3):

  estimator  = bcn.BCNClassifier(B = int(B),
                                 nu = nu,
                                 lam = lam,
                                 r = r,
                                 tol = tol,
                                 col_sample = col_sample,
                                 n_clusters = n_clusters,
                                 activation="tanh",
                                 type_optim="nlminb",
                                 show_progress = False)

  return -cross_val_score(estimator, X_train, y_train,
                          scoring='f1_macro',
                          cv=5, n_jobs=None,
                          verbose=0).mean()

def optimize_bcn(X_train, y_train):
  # objective function for hyperparams tuning
  def crossval_objective(x):
    return bcn_cv(X_train=X_train,
                  y_train=y_train,
                  B = int(x[0]),
                  nu = 10**x[1],
                  lam = 10**x[2],
                  r = 1 - 10**x[3],
                  tol = 10**x[4],
                  col_sample = np.ceil(x[5]),
                  n_clusters = np.ceil(x[6]))
  gp_opt = gp.GPOpt(objective_func=crossval_objective,
                    lower_bound = np.array([   3,    -6, -10, -10,   -6, 0.8, 1]),
                    upper_bound = np.array([ 100,  -0.1,  10,  -1, -0.1,   1, 4]),
                    min_value = -1.0, 
                    n_init=10, n_iter=50, seed=3137)
  return {'parameters': gp_opt.lazyoptimize(method="mc",
                                            estimators = ["BaggingRegressor", 
                                                          "ExtraTreesRegressor", 
                                                          "RandomForestRegressor"],
                                            verbose=2, abs_tol=1e-1), 'opt_object':  gp_opt}

In [7]:
start = time()
for elt in datasets:

  dataset = elt()
  X = dataset.data
  y = dataset.target

  # split data into training test and test set
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=0.2, random_state=3137)

  # hyperparams tuning
  res1 = optimize_bcn(X_train, y_train)
  print(res1)
  parameters = res1["parameters"]
  estimator = bcn.BCNClassifier(B=int(parameters[0][0]),
                                nu=10**parameters[0][1],
                                lam=10**parameters[0][2],
                                r=1-10**parameters[0][3],
                                tol=10**parameters[0][4],
                                col_sample=np.ceil(parameters[0][5]),
                                n_clusters=np.ceil(parameters[0][6]),
                                activation="tanh",
                                type_optim="nlminb").fit(X_train, y_train)
  print(f"\n\n Test set accuracy: {estimator.score(X_test, y_test)}") 
  print(f"\n Elapsed: {time() - start}") 


 adjusting surrogate model # 1 (CustomRegressor(BaggingRegressor))... 


 adjusting surrogate model # 1 (CustomRegressor(BaggingRegressor))... 


 Creating initial design... 


 ...Done. 


 Optimization loop... 

iteration 1 -----
current minimum:  [87.875  -0.8375 -7.5    -4.375  -3.7875  0.875   2.875 ]
current minimum score:  -0.9467825589704711
next parameter: [29.25997925 -0.90249939 -1.29089355 -7.03671265 -5.42616882  0.99440308
  1.83303833]
score for next parameter: -1.0 

{'parameters': DescribeResult(best_params=array([29.25997925, -0.90249939, -1.29089355, -7.03671265, -5.42616882,
        0.99440308,  1.83303833]), best_score=-1.0, best_surrogate=CustomRegressor(obj=BaggingRegressor(), replications=150, type_pi='kde')), 'opt_object': <GPopt.GPOpt.GPOpt object at 0x1328c7610>}
  |======================================================================| 100%


 Test set accuracy: 0.9722222222222222

 Elapsed: 53.765836000442505

 adjusting surrogate model # 1 (CustomRegresso

## 2 - 2 - bootstrap surrogates

In [8]:
def bcn_cv(X_train, y_train,
               B = 10, nu = 0.335855,
               lam = 10**0.7837525,
               r = 1 - 10**(-5.470031),
               tol = 10**-7,
               col_sample=1,
               n_clusters = 3):

  estimator  = bcn.BCNClassifier(B = int(B),
                                 nu = nu,
                                 lam = lam,
                                 r = r,
                                 tol = tol,
                                 col_sample = col_sample,
                                 n_clusters = n_clusters,
                                 activation="tanh",
                                 type_optim="nlminb",
                                 show_progress = False)

  return -cross_val_score(estimator, X_train, y_train,
                          scoring='f1_macro',
                          cv=5, n_jobs=None,
                          verbose=0).mean()

def optimize_bcn2(X_train, y_train):
  # objective function for hyperparams tuning
  def crossval_objective(x):
    return bcn_cv(X_train=X_train,
                  y_train=y_train,
                  B = int(x[0]),
                  nu = 10**x[1],
                  lam = 10**x[2],
                  r = 1 - 10**x[3],
                  tol = 10**x[4],
                  col_sample = np.ceil(x[5]),
                  n_clusters = np.ceil(x[6]))
  gp_opt = gp.GPOpt(objective_func=crossval_objective,
                    lower_bound = np.array([   3,    -6, -10, -10,   -6, 0.8, 1]),
                    upper_bound = np.array([ 100,  -0.1,  10,  -1, -0.1,   1, 4]),
                    min_value = -1.0, 
                    n_init=10, n_iter=50, seed=3137)
  return {'parameters': gp_opt.lazyoptimize(method="mc",
                                            type_pi="bootstrap",
                                            estimators = ["BaggingRegressor", 
                                                          "ExtraTreesRegressor", 
                                                          "RandomForestRegressor"],
                                            verbose=2, abs_tol=1e-1), 'opt_object':  gp_opt}

In [9]:
start = time()
for elt in datasets:

  dataset = elt()
  X = dataset.data
  y = dataset.target

  # split data into training test and test set
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=0.2, random_state=3137)

  # hyperparams tuning
  res1 = optimize_bcn2(X_train, y_train)
  print(res1)
  parameters = res1["parameters"]

  estimator = bcn.BCNClassifier(B=int(parameters[0][0]),
                                nu=10**parameters[0][1],
                                lam=10**parameters[0][2],
                                r=1-10**parameters[0][3],
                                tol=10**parameters[0][4],
                                col_sample=np.ceil(parameters[0][5]),
                                n_clusters=np.ceil(parameters[0][6]),
                                activation="tanh",
                                type_optim="nlminb").fit(X_train, y_train)
  print(f"\n\n Test set accuracy: {estimator.score(X_test, y_test)}")
  print(f"\n Elapsed: {time() - start}")


 adjusting surrogate model # 1 (CustomRegressor(BaggingRegressor))... 


 adjusting surrogate model # 1 (CustomRegressor(BaggingRegressor))... 


 Creating initial design... 


 ...Done. 


 Optimization loop... 

iteration 1 -----
current minimum:  [87.875  -0.8375 -7.5    -4.375  -3.7875  0.875   2.875 ]
current minimum score:  -0.9467825589704711
next parameter: [25.25482178 -0.68589478 -8.78540039 -5.40057373 -4.60890503  0.93834229
  1.52459717]
score for next parameter: -0.9115434958743615 

iteration 2 -----
current minimum:  [87.875  -0.8375 -7.5    -4.375  -3.7875  0.875   2.875 ]
current minimum score:  -0.9467825589704711
next parameter: [21.75289917 -1.17185974 -8.58093262 -1.59408569 -3.98429871  0.8882019
  1.14968872]
score for next parameter: -0.8703990911365164 

iteration 3 -----
current minimum:  [87.875  -0.8375 -7.5    -4.375  -3.7875  0.875   2.875 ]
current minimum score:  -0.9467825589704711
next parameter: [ 6.02532959 -1.26782837 -5.56274414 -2.94842529 -5.06